### Initiate session

In [1]:
#Open Session
from simulator_interface import open_session, close_session
#assigning variable names for the session
simulator, wolf, sheep = open_session(n_epucks=2)


In [ ]:
close_session(simulator) # Just in case something goes wrong, in order to restart the session, execute this cell and select also Kernel - Restart and then open session again

### Defining basic behaviors (fear, aggression, obstacle avoidance)

In [2]:
#Defining fear behavior -- if other ePuck comes near, accelerate speed to get away from other ePuck
def fear(robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    left_wheel = left + 2
    right_wheel = right + 2
    return left_wheel, right_wheel

#Defining aggressive behavior -- if other ePuck comes near, accelerate speed towards other ePuck
def aggression(robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    left_wheel = right + 1 
    right_wheel = left + 1
    return left_wheel, right_wheel

#Defining obstacle avoidance behavior -- if the proximity sensors register a wall or a tree, ePuck turns away
def obstacle_avoidance(robot):
    left, right = robot.prox_activations(tracked_objects=["20cmHighWall", "Tree"])  
    sum_of_proxs = left + right
    wheel_activation_1 = 1.0 - sum_of_proxs / 2.0
    wheel_activation_2 = 1.0 - sum_of_proxs / 1.5  
    return wheel_activation_1, wheel_activation_2

### Define behavior so epucks only seek out food when hungry

In [3]:
wolf.has_eaten() #Tells us if the wolf robot has eaten a sphere since the last call of the function

False

In [4]:
# Starting sphere apparition in the environment:
simulator.start_sphere_apparition(period=3)

Routine sphere_apparition started
Routine eating started


In [5]:
#A function to compute the glucose level of the robot
def foraging_drive(robot): 
    if robot.has_eaten():
        robot.glucose_level += 0.2  # if the robot has eaten a sphere, increase its glucose level by 0.2
    else:
        robot.glucose_level -= 0.03  # otherwise (nothing eaten), decrease the glucose level by 0.03
    
    # The line below bounds the value of the glucose level between 0 and 1
    robot.glucose_level = min(1., max(robot.glucose_level, 0.))

#Defining behaviors for the robot to search for food when hungry/ glucose level low (foraging) 
def foraging(robot):
    left, right = robot.prox_activations(tracked_objects=["Sphere"])
    left_activation = right
    right_activation = left
    return left_activation, right_activation, 1 - robot.glucose_level

#Defining behavior to neglect food when full/ glucose level high (neglecting)    
def neglecting(robot):
    left, right = robot.prox_activations(tracked_objects=["Sphere"])
    left_activation = left
    right_activation = right
    return left_activation, right_activation, robot.glucose_level

In [6]:
#Defining initial glucose level for wolf and sheep robot (wolf is more hungry at the beginning)
wolf.glucose_level = 0.5
sheep.glucose_level = 0.8

### Attach and start all behaviors and routines

In [7]:
wolf.detach_all_behaviors()
sheep.detach_all_behaviors()

# Then we attach the desired behaviors to epuck1/ the sheep:
sheep.attach_behavior(fear, freq=10)
sheep.attach_behavior(obstacle_avoidance, freq=10)
sheep.attach_behavior(foraging, freq=10)
sheep.attach_behavior(neglecting, freq=10)
sheep.attach_routine(foraging_drive, freq=1)
# Then we attach also the corresponding behaviors to epuck2/ the wolf:
wolf.attach_behavior(aggression, freq=10)
wolf.attach_behavior(obstacle_avoidance, freq=10)
wolf.attach_behavior(foraging, freq=10)
wolf.attach_behavior(neglecting, freq=10)
wolf.attach_routine(foraging_drive, freq=1)

sheep.start_all_behaviors()  # This will start all attached behaviors (fear, obstacle_avoidance etc.) for sheep
wolf.start_all_behaviors()  # This will start all attached behaviors (aggression, obstacle_avoidance etc.) for wolf
wolf.start_routine(foraging_drive) # This will start the foraging_drive routine for wolf
sheep.start_routine(foraging_drive) # This will start the foraging_drive routine for sheep

Behavior fear started
Behavior obstacle_avoidance started
Behavior foraging started
Behavior neglecting started
Behavior aggression started
Behavior obstacle_avoidance started
Behavior foraging started
Behavior neglecting started
Routine foraging_drive started
Routine foraging_drive started
